In [ ]:
import sys
import pyemu
from pathlib import Path
import shutil, os, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#### PyCap Run Name is what all your outputs will have as a name. 
pycap_run_name = "LPR_Redux"

#### Base directory for runs
parent_run_path = Path("../pycap_runs")

#### depletion potential calculations directory
base_run_path = parent_run_path / "pycap_base"
pest_path = parent_run_path / "pycap_pest"
template_path = pest_path / "pycap_template"

#### finally define the scripts directory
script_path = Path("../scripts")

In [ ]:
# Copy over directories

MASTER_path = pest_path / "MASTER"
if MASTER_path.exists():
    shutil.rmtree(MASTER_path)
# copy over the binaries into template first so they get distributed


#copy over the correct binary
if "window" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/windows/pestpp-ies.exe', template_path / 'pestpp-ies.exe')
elif "linux" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/linux/pestpp-ies', template_path / 'pestpp-ies')
elif "mac" in platform.platform().lower():
    shutil.copy2('../../binaries/PESTPP/mac/pestpp-ies', template_path / 'pestpp-ies')

# we also need the forward run script
shutil.copy2(script_path / 'run_pycap_standalone.py', 
             template_path / 'run_pycap_standalone.py')

# and we need the base yml file
shutil.copy2(base_run_path / 'LPR_Redux.yml',
              template_path / 'LPR_Redux.yml')

# finally populate the NOPTMAX=0 and MASTER paths with the template_path files
shutil.copytree(template_path, MASTER_path)
os.remove(MASTER_path / 'allobs.out')



In [ ]:
pyemu.os_utils.run('pestpp-ies prior_mc.pst /e', cwd=MASTER_path)


### Post-processing Monte Carlo Results


In [ ]:
# check phi
phi = pd.read_csv('../pycap_runs/pycap_pest/MASTER/prior_mc.phi.actual.csv',index_col=0).T.iloc[5:]
phi_org = phi.copy()
phi.columns =['phi']
phi.hist(bins=50)

# only change if there are a very small number of extreme outliers
phitoohigh = 20000000
phi = phi.loc[phi.phi<phitoohigh]
phi.hist(bins=50)

reals_to_keep = phi.index
reject_reals = list(set(phi_org.index) - set(reals_to_keep)) # Keep rejected realizations in case we want to review them
print('Number of MC runs kept is ', len(reals_to_keep))


# read in all MC observations
# Note: Large file, takes a while to read in. 
obs = pd.read_csv('../pycap_runs/pycap_pest/MASTER/prior_mc.0.obs.csv', index_col=0)
obs.index = obs.index.astype(str)

obs = obs.loc[reals_to_keep] 

# parse into time-series
ts_obs = obs[[i for i in obs.columns if not i.endswith(':bdpl')]]
bdpl_obs = obs[[i for i in obs.columns if i.endswith(':bdpl')]]

ts_lox = np.unique([i.split('__')[0] for i in ts_obs.columns])

tmp2 = ts_obs.T.copy()
tmp2['time'] = [int(i.split('__')[1]) for i in tmp2.index]
tmp2['lox'] = [str(i.split('__')[0]) for i in tmp2.index]

# look at timeseries outputs
all_time_series = pd.read_csv('../pycap_runs/pycap_base/output/{}.table_report.all_ts.csv'.format(pycap_run_name), index_col=0)

# sum up cumulative depletions (NOTE: assumes only one stream used in MC)
cum_depl = all_time_series.sum(axis=1)

# plot up time-series depletions
fig,ax = plt.subplots(nrows=1, ncols=1)
ax.plot(all_time_series, alpha=0.6, c='k', lw=0.2)
ax.plot(cum_depl, c='r')
#fig.set_title('Cumulative Depl in all streams in Monte Carlo Analysis')
plt.savefig('../pycap_runs/pycap_pest/MASTER/cum_depl_MC_{}.png'.format(pycap_run_name))




In [ ]:
# plot up each well's last year of depletion
# NOTE: takes a while to run.

fig,ax = plt.subplots(nrows=len(ts_lox), ncols=1, sharex=True, figsize=(10,4))
i=0
            
for cts in ts_lox:
    tmp = tmp2.loc[tmp2['lox']==cts]
    tmp.set_index('time', drop = True, inplace=True)
    tmp = tmp.drop('lox', axis=1)
    ax =tmp.plot(alpha=0.4, c='k', lw=0.1, legend=None)
    tmp['base'].plot(c='r')
    ax.set_title(cts)
    i+=1
plt.tight_layout()
plt.savefig('../pycap_runs/pycap_pest/MASTER/individual_depl_MC_{}.png'.format(pycap_run_name))